## The pourpose of this notebook is to implement a simple classification model and show how an exported ONNX could be use to distribute an model already trained to be use by other applications.

In [68]:
}import torch
import torch.nn as nn
import torch.optim as optim
# import torch.onnx as onnx
import numpy as np


define the hyperparameters of the model:

In [58]:
learning_rate = 0.001
num_epochs = 1000


define the model architecture. We'll use a simple fully connected neural network with one hidden layer and a sigmoid activation function at the output

In [59]:
class ClarificationModel(nn.Module):
    def __init__(self):
        super(ClarificationModel, self).__init__()
        self.fc1 = nn.Linear(3, 5)
        self.fc2 = nn.Linear(5, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.sigmoid(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x


define a function to generate the training data. We'll generate 1000 random RGB values with integer values ranging from 0 to 255, and set the label to 1 if the color is some tone of orange (i.e., if the R value is greater than the G and B values).

In [60]:
def generate_data(num_samples):
    X = np.random.randint(0, 256, size=(num_samples, 3))
    y = np.zeros((num_samples, 1))
    for i in range(num_samples):
        if X[i, 0] > X[i, 1] and X[i, 0] > X[i, 2]:
            y[i] = 1
    return X, y


train the model. We'll use the binary cross entropy loss and the Adam optimizer.

In [61]:
model = ClarificationModel()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

X, y = generate_data(100000)

for epoch in range(num_epochs):
    optimizer.zero_grad()
    inputs = torch.from_numpy(X).float()
    labels = torch.from_numpy(y).float()
    outputs = model(inputs / 255.0)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [1/1000], Loss: 0.6635
Epoch [101/1000], Loss: 0.6378
Epoch [201/1000], Loss: 0.6295
Epoch [301/1000], Loss: 0.6207
Epoch [401/1000], Loss: 0.6067
Epoch [501/1000], Loss: 0.5862
Epoch [601/1000], Loss: 0.5597
Epoch [701/1000], Loss: 0.5288
Epoch [801/1000], Loss: 0.4958
Epoch [901/1000], Loss: 0.4632


se the trained model to make predictions on new data.

In [62]:
def predict_color(model, color):
    inputs = torch.from_numpy(np.array(color)).float()
    output = model(inputs / 255.0)
    if output.item() > 0.5:
        return 'Some tone of orange'
    else:
        return 'Not some tone of orange'

print(predict_color(model, [255, 128, 0]))  # Some tone of orange
print(predict_color(model, [128, 255, 0]))  # Not some tone of orange
print(predict_color(model, [223, 118, 14]))  # Some tone of orange

Some tone of orange
Not some tone of orange
Some tone of orange


export the model to ONNX

In [63]:
# # Export the model to ONNX
input_names = ["input"]
output_names = ["output"]
dummy_input = torch.randn(1, 3)
torch.onnx.export(model, dummy_input, "classifier.onnx", input_names=input_names, output_names=output_names)

# Using the exported ONNX

In [64]:
!pip install onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


loads an ONNX model and makes predictions on input data:

In [65]:
import onnxruntime as ort
import numpy as np

def predict_onnx(model_file, input_data):
    # Load the ONNX model
    session = ort.InferenceSession(model_file)

    # Get the names of the input and output tensors
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name

    # Run the model on the input data
    output_data = session.run([output_name], {input_name: input_data})[0]

    # Return the output of the model
    if output_data > 0.5:
        return "Some tone of orange"
    else:
        return "Not some tone of orange"


test an orange-ish color

In [66]:
input_data = np.array([[255, 128, 0]], dtype=np.float32)
output = predict_onnx("classifier.onnx", input_data)
print(output)

Some tone of orange


test a non oranh-ish color

In [67]:
input_data = np.array([[128, 255, 0]], dtype=np.float32)
output = predict_onnx("classifier.onnx", input_data)
print(output)

Not some tone of orange
